In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import TargetEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn import set_config
set_config(enable_metadata_routing=True)

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from time import monotonic

from utils.data_processing import load_data, raw_columns, full_dtypes, transform_datetime, df_ua_parser, transform_ipinfo, transform_packetinfo, transform_proxyinfo

# Input Data Processing

In [2]:
RANDOM_STATE = 124
data_path = Path("./data")
if data_path.joinpath("first_ml_processing.csv").exists():
    processed_data = pd.read_csv(data_path.joinpath("first_ml_processing.csv"))
    raw_data = pd.read_csv(data_path.joinpath("cybersecurity_attacks.csv"))
    
    ip_cols = ["Int Source IP", "Int Destination IP", "Global Source IP", "Global Destination IP"]
    raw_data[ip_cols] = transform_ipinfo(raw_data[["Source IP Address", "Destination IP Address"]])
else:
    # Must use clean_data function to load data 
    dtypes = {col: col_type for col, col_type in full_dtypes.items() if col in raw_columns}
    raw_data = load_data(data_path.joinpath("cybersecurity_attacks.csv"), dtype=dtypes)

    datetime_cols = ["Year", "Month", "Day", "Hour", "Minute", "Second", "DayOfWeek", "IsWeekend"]
    raw_data[datetime_cols] = transform_datetime(raw_data["Timestamp"])
    device_cols = ["String","Browser Name", "Browser Version", "Browser Minor", "Browser Patch",
                    "Browser Patch Minor", "OS Name", "OS Version", "OS Version Minor",
                    "OS Version Patch", "OS Version Patch Minor", "Device Brand", "Device Model",
                    "Device Type"]
    raw_data[device_cols] = df_ua_parser(raw_data["Device Information"])
    proxy_cols = ["Is Proxy"]
    raw_data[proxy_cols] = transform_proxyinfo(raw_data["Proxy Information"])
    ip_cols = ["Int Source IP", "Int Destination IP", "Global Source IP", "Global Destination IP"]
    raw_data[ip_cols] = transform_ipinfo(raw_data[["Source IP Address", "Destination IP Address"]])
    packet_cols = ["Packet Bin"]
    raw_data[packet_cols] = transform_packetinfo(raw_data["Packet Length"], scale=False)

    processed_data = raw_data.drop(columns=["Payload Data","Timestamp", "String", "Device Information",
                                    "Proxy Information", "Source IP Address", "Destination IP Address"])
    processed_data.to_csv(data_path.joinpath("first_ml_processing.csv"), index=False)


# Baseline Model
The baseline model considers all features. <br>
Numerical values are kept that way and all others features are one-hot encoded.


In [3]:
Y_true = processed_data["Attack Type"].astype("category").cat.codes
labels = ["DDoS", "Intrusion", "Malware"]
X_dataset = raw_data.drop(columns=["Attack Type", "Global Source IP", "Global Destination IP", "Source IP Address", "Destination IP Address"])

In [ ]:
cat_cols = X_dataset.select_dtypes(include=["str", "category"]).columns
print("Categorical columns:", cat_cols.tolist())
bool_cols = X_dataset.select_dtypes(include=["bool"]).columns
print("Boolean columns:", bool_cols.tolist())
num_cols = X_dataset.select_dtypes(include=["int64", "float64"]).columns
print("Numerical columns:", num_cols.tolist())
passthrough_cols = [col for col in X_dataset.columns if col not in cat_cols and col not in bool_cols and col not in num_cols]

_tmp = SimpleImputer(strategy="constant", fill_value="unknown").fit_transform(X_dataset[cat_cols])
_tmp_ohe = OneHotEncoder(drop="first", handle_unknown="ignore").fit(_tmp)
all_categories = _tmp_ohe.categories_

## Baseline Comparison

In another script the model seems to give a high recall for a specific attack type. That could explain why smote work because then it would create more data for the other 2 categories

### Pipeline

In [ ]:
numeric_transformer_all = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler())
    ]
)

cat_transformer_all = Pipeline([
    ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant", fill_value="unknown")),
    ("encoder", OneHotEncoder(drop="first", handle_unknown="ignore", sparse_output=False, categories=all_categories))
])

preprocessor_all = ColumnTransformer(
    transformers=[
        ("cat", cat_transformer_all, cat_cols),
        ("num", numeric_transformer_all, num_cols)
    ]
)

classifier_all = RandomForestClassifier(
    random_state=RANDOM_STATE, 
    n_estimators=200, 
    max_depth=20, 
    min_samples_split=5,
    n_jobs=6,
    verbose=1
)

base_model = Pipeline(
    steps=[
        ("preprocessor", preprocessor_all),
        ("classifier", classifier_all)
    ]
)


Time taken to fit the model: 3.22 seconds


/home/florians/cours/CyberSecurity_ML/.venv/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:261: UserWarning:

Found unknown categories in columns [0, 4, 10, 11, 13, 14] during transform. These unknown categories will be encoded as all zeros



Model score: 0.337
              precision    recall  f1-score   support

        DDoS       0.58      0.33      0.42      4649
   Intrusion       0.15      0.35      0.21      1153
     Malware       0.28      0.34      0.31      2198

    accuracy                           0.34      8000
   macro avg       0.34      0.34      0.31      8000
weighted avg       0.43      0.34      0.36      8000



/home/florians/cours/CyberSecurity_ML/.venv/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:261: UserWarning:

Found unknown categories in columns [0, 4, 10, 11, 13, 14] during transform. These unknown categories will be encoded as all zeros



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, Y_true, test_size=0.2, stratify=Y_true, random_state=RANDOM_STATE)
start_time = monotonic()
base_model.fit(X_train, y_train)
print("Time taken to fit the model: %.2f seconds" % (monotonic() - start_time))
print("Model score: %.3f" % base_model.score(X_test, y_test))
y_pred = base_model.predict(X_test)
print(classification_report(y_pred, y_test, target_names=labels))

### Feature Importance

In [ ]:
importances = base_model.named_steps['classifier'].feature_importances_
std = np.std([tree.feature_importances_ for tree in base_model.named_steps['classifier'].estimators_], axis=0)
feature_names = [f"{x}" for x in base_model.named_steps["preprocessor"].get_feature_names_out()]
forest_importances = pd.DataFrame({"feature": feature_names, "importance": importances, "std": std}).sort_values("importance", ascending=False)

fig = px.bar(forest_importances[forest_importances["importance"] > 0.001],x="feature",y="importance", error_y="std", title="Feature importances using MDI")
fig.show()

## Restrained Categories

### Pipeline Definition

In [ ]:

numeric_transformer = Pipeline(
        steps = [("scaler", StandardScaler())]
    )

categorical_transformer = Pipeline([
        ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant", fill_value="unknown")),
        ("encoder", OneHotEncoder(handle_unknown="ignore", drop="first", sparse_output=True, categories=40)) #Limit categories to 40 to prevent too much indiviual features after encoding
        ])
boolean_transformer = Pipeline([
        ("encoder", TargetEncoder(target_type="binary")),
        ])

preprocessor = ColumnTransformer(
        transformers=[
            ("cat", categorical_transformer, cat_cols),
            ("num", numeric_transformer, num_cols),
            ("bool", boolean_transformer, bool_cols)
        ]
    )

cstr_model = Pipeline([
        ("preprocessor", preprocessor),
        ("classifier", RandomForestClassifier(n_estimators=200, random_state=RANDOM_STATE, n_jobs=6, verbose=1))
    ])

Categorical columns: ['Timestamp', 'Protocol', 'Packet Type', 'Traffic Type', 'Payload Data', 'Malware Indicators', 'Alerts/Warnings', 'Attack Signature', 'Action Taken', 'Severity Level', 'User Information', 'Device Information', 'Network Segment', 'Geo-location Data', 'Proxy Information', 'Firewall Logs', 'IDS/IPS Alerts', 'Log Source']
Boolean columns: []
Numerical columns: ['Source Port', 'Destination Port', 'Packet Length', 'Anomaly Scores', 'Int Source IP', 'Int Destination IP']


### Model Fitting and Result

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, Y_true, test_size=0.2, stratify=Y_true, random_state=RANDOM_STATE)
start_time = monotonic()
cstr_model.fit(X_train, y_train)
print("Time taken to fit the model: %.2f seconds" % (monotonic() - start_time))
print("Model score: %.3f" % cstr_model.score(X_test, y_test))
y_pred = cstr_model.predict(X_test)
print(classification_report(y_pred, y_test, target_names=labels))

[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:   33.7s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:  2.5min
[Parallel(n_jobs=6)]: Done 200 out of 200 | elapsed:  2.6min finished


Time taken to fit the model: 158.29 seconds


[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.1s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:    0.5s
[Parallel(n_jobs=6)]: Done 200 out of 200 | elapsed:    0.6s finished


Model score: 0.324


[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.1s


              precision    recall  f1-score   support

        DDoS       0.40      0.32      0.36      3350
   Intrusion       0.28      0.33      0.30      2274
     Malware       0.29      0.32      0.30      2376

    accuracy                           0.32      8000
   macro avg       0.32      0.32      0.32      8000
weighted avg       0.33      0.32      0.33      8000



[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:    0.5s
[Parallel(n_jobs=6)]: Done 200 out of 200 | elapsed:    0.6s finished


### Feature Importance

In [ ]:
importances = cstr_model.named_steps['classifier'].feature_importances_
std = np.std([tree.feature_importances_ for tree in cstr_model.named_steps['classifier'].estimators_], axis=0)
feature_names = [f"{x}" for x in cstr_model.named_steps["preprocessor"].get_feature_names_out()]
forest_importances = pd.DataFrame({"feature": feature_names, "importance": importances, "std": std}).sort_values("importance", ascending=False)

fig = px.bar(forest_importances[forest_importances["importance"] > 0.001],x="feature",y="importance", error_y="std", title="Feature importances using MDI")
fig.show()

## No pre-defined categories

In [ ]:
numeric_transformer_all = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler())
    ]
)

cat_transformer_all = Pipeline([
    ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant", fill_value="unknown")),
    ("encoder", OneHotEncoder(drop="first", handle_unknown="ignore", sparse_output=True))
])

preprocessor_all = ColumnTransformer(
    transformers=[
        ("cat", cat_transformer_all, cat_cols),
        ("num", numeric_transformer_all, num_cols)
    ]
)

classifier_all = RandomForestClassifier(
    random_state=RANDOM_STATE, 
    n_estimators=200, 
    max_depth=20, 
    min_samples_split=5,
    n_jobs=6,
    verbose=1
)

comp_model = Pipeline(
    steps=[
        ("preprocessor", preprocessor_all),
        ("classifier", classifier_all)
    ]
)

start_time = monotonic()
comp_model.fit(X_train, y_train)
print("Time taken to fit the model: %.2f seconds" % (monotonic() - start_time))
print("Model score: %.3f" % comp_model.score(X_test, y_test))
y_pred = comp_model.predict(X_test)
print(classification_report(y_pred, y_test, target_names=labels))

[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.8s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:    3.7s
[Parallel(n_jobs=6)]: Done 200 out of 200 | elapsed:    3.9s finished


Time taken to fit the model: 4.62 seconds


[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:    0.1s
[Parallel(n_jobs=6)]: Done 200 out of 200 | elapsed:    0.1s finished


Model score: 0.338
              precision    recall  f1-score   support

        DDoS       0.88      0.34      0.49      7030
   Intrusion       0.06      0.37      0.10       438
     Malware       0.06      0.32      0.11       532

    accuracy                           0.34      8000
   macro avg       0.34      0.34      0.23      8000
weighted avg       0.78      0.34      0.44      8000



[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:    0.1s
[Parallel(n_jobs=6)]: Done 200 out of 200 | elapsed:    0.1s finished


Huge precision according to DDoS but rest is very low. Means using smote/smotetomek need to balance the synthetic data to more Intrusion & Malware!! <br> try to remove payload and timestamp maybe

In [ ]:
importances = comp_model.named_steps['classifier'].feature_importances_
std = np.std([tree.feature_importances_ for tree in comp_model.named_steps['classifier'].estimators_], axis=0)
feature_names = [f"{x}" for x in comp_model.named_steps["preprocessor"].get_feature_names_out()]
forest_importances = pd.DataFrame({"feature": feature_names, "importance": importances, "std": std}).sort_values("importance", ascending=False)

fig = px.bar(forest_importances[forest_importances["importance"] > 0.001],x="feature",y="importance", error_y="std", title="Feature importances using MDI")
fig.show()

# Basic Models

## Model with all granular features

Because we are not combining features, all numerical features with missing values will not be considered. There is no method to apply a specific method to replace it for features like OS version patch minor or browser patch minor without getting extra data <br>
All granular is used and we removed all unnecessary features from the raw file

In [10]:
X_dataset = processed_data.copy().drop(columns=["Attack Type", "Browser Patch" , "Browser Patch Minor",
                                                "OS Version", "OS Version Minor", "OS Version Patch", "OS Version Patch Minor",
                                                "Device Type", "User Information", "Geo-location Data"])

### Pipeline Definition

In [11]:
cat_cols = X_dataset.select_dtypes(include=["str", "category"]).columns
print("Categorical columns:", cat_cols.tolist())
bool_cols = X_dataset.select_dtypes(include=["bool"]).columns
print("Boolean columns:", bool_cols.tolist())
num_cols = X_dataset.select_dtypes(include=["int64", "float64"]).columns
print("Numerical columns:", num_cols.tolist())
passthrough_cols = [col for col in X_dataset.columns if col not in cat_cols and col not in bool_cols and col not in num_cols]

numeric_transformer = Pipeline(
        steps = [("scaler", StandardScaler())]
    )

categorical_transformer = Pipeline([
        ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant", fill_value="unknown")),
        ("encoder", OneHotEncoder(handle_unknown="ignore", drop="first"))
        ])
boolean_transformer = Pipeline([
        ("encoder", TargetEncoder(target_type="binary")),
        ])

preprocessor = ColumnTransformer(
        transformers=[
            ("cat", categorical_transformer, cat_cols),
            ("num", numeric_transformer, num_cols),
            ("bool", boolean_transformer, bool_cols)
        ]
    )

first_model = Pipeline([
        ("preprocessor", preprocessor),
        ("classifier", RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE, n_jobs=6, verbose=1))
    ])

Categorical columns: ['Protocol', 'Packet Type', 'Traffic Type', 'Malware Indicators', 'Alerts/Warnings', 'Attack Signature', 'Action Taken', 'Severity Level', 'Network Segment', 'Firewall Logs', 'IDS/IPS Alerts', 'Log Source', 'Browser Name', 'OS Name', 'Device Brand', 'Device Model', 'Packet Bin']
Boolean columns: ['Is Proxy', 'Global Source IP', 'Global Destination IP']
Numerical columns: ['Source Port', 'Destination Port', 'Packet Length', 'Anomaly Scores', 'Year', 'Month', 'Day', 'Hour', 'Minute', 'Second', 'DayOfWeek', 'IsWeekend', 'Browser Version', 'Browser Minor', 'Int Source IP', 'Int Destination IP']


### Model Fitting and Result

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, Y_true, test_size=0.2, stratify=Y_true, random_state=RANDOM_STATE)
start_time = monotonic()
first_model.fit(X_train, y_train)
print("Time taken to fit the model: %.2f seconds" % (monotonic() - start_time))
print("Model score: %.3f" % first_model.score(X_test, y_test))
y_pred = first_model.predict(X_test)
print(classification_report(y_pred, y_test, target_names=labels))

[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.5s
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed:    1.2s finished


Time taken to fit the model: 1.47 seconds
Model score: 0.326
              precision    recall  f1-score   support

        DDoS       0.37      0.33      0.35      3075
   Intrusion       0.31      0.33      0.32      2465
     Malware       0.30      0.32      0.31      2460

    accuracy                           0.33      8000
   macro avg       0.33      0.33      0.32      8000
weighted avg       0.33      0.33      0.33      8000



[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed:    0.0s finished


So nothing new. We obtain a 1/3 which is expected from randomness

### Feature Importance

In [13]:
importances = first_model.named_steps['classifier'].feature_importances_
std = np.std([tree.feature_importances_ for tree in first_model.named_steps['classifier'].estimators_], axis=0)
feature_names = [f"{x}" for x in first_model.named_steps["preprocessor"].get_feature_names_out()]
forest_importances = pd.DataFrame({"feature": feature_names, "importance": importances, "std": std}).sort_values("importance", ascending=False)

fig = px.bar(forest_importances,x="feature",y="importance", error_y="std", title="Feature importances using MDI")
fig.show()


Each feature show very low importance (most below 5%) but all numerical features seems to carry the most information

## 2nd model: Numeric as categories


Some numeric features are considered as categories now. All features related to date and time information as well as the one related to browser and device version

In [14]:
Xcat_dataset = X_dataset.copy()
columns = ["Year", "Month", "Day", "Hour", "Minute", "Second", "DayOfWeek", "Browser Version", "Browser Minor"]
Xcat_dataset[columns] = Xcat_dataset[columns].astype("category")

### Pipeline Definition

In [15]:
cat_cols = Xcat_dataset.select_dtypes(include=["str", "category"]).columns
print("Categorical columns:", cat_cols.tolist())
bool_cols = Xcat_dataset.select_dtypes(include=["bool"]).columns
print("Boolean columns:", bool_cols.tolist())
num_cols = Xcat_dataset.select_dtypes(include=["int64", "float64"]).columns
print("Numerical columns:", num_cols.tolist())
passthrough_cols = [col for col in Xcat_dataset.columns if col not in cat_cols and col not in bool_cols and col not in num_cols]

numeric_transformer = Pipeline(
        steps = [("scaler", StandardScaler())]
    )

categorical_transformer = Pipeline([
        ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant", fill_value="unknown")),
        ("encoder", OneHotEncoder(handle_unknown="ignore", drop="first"))
        ])
boolean_transformer = Pipeline([
        ("encoder", TargetEncoder(target_type="binary")),
        ])

preprocessor = ColumnTransformer(
        transformers=[
            ("cat", categorical_transformer, cat_cols),
            ("num", numeric_transformer, num_cols),
            ("bool", boolean_transformer, bool_cols)
        ]
    )

cat_model = Pipeline([
        ("preprocessor", preprocessor),
        ("classifier", RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE, n_jobs=6, verbose=1))
    ])

Categorical columns: ['Protocol', 'Packet Type', 'Traffic Type', 'Malware Indicators', 'Alerts/Warnings', 'Attack Signature', 'Action Taken', 'Severity Level', 'Network Segment', 'Firewall Logs', 'IDS/IPS Alerts', 'Log Source', 'Year', 'Month', 'Day', 'Hour', 'Minute', 'Second', 'DayOfWeek', 'Browser Name', 'Browser Version', 'Browser Minor', 'OS Name', 'Device Brand', 'Device Model', 'Packet Bin']
Boolean columns: ['Is Proxy', 'Global Source IP', 'Global Destination IP']
Numerical columns: ['Source Port', 'Destination Port', 'Packet Length', 'Anomaly Scores', 'IsWeekend', 'Int Source IP', 'Int Destination IP']


### Model Fitting and Result

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, Y_true, test_size=0.2, stratify=Y_true, random_state=RANDOM_STATE)
start_time = monotonic()
cat_model.fit(X_train, y_train)
print("Time taken to fit the model: %.2f seconds" % (monotonic() - start_time))
print("Model score: %.3f" % cat_model.score(X_test, y_test))
y_pred = cat_model.predict(X_test)
print(classification_report(y_pred, y_test, target_names=labels))

[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    3.1s
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed:    7.7s finished


Time taken to fit the model: 8.01 seconds
Model score: 0.335
              precision    recall  f1-score   support

        DDoS       0.38      0.35      0.36      2925
   Intrusion       0.30      0.32      0.31      2493
     Malware       0.32      0.33      0.33      2582

    accuracy                           0.33      8000
   macro avg       0.33      0.33      0.33      8000
weighted avg       0.34      0.33      0.34      8000



[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed:    0.0s finished


Not better. Same score and no improvement

### Feature Importance

In [17]:
importances = cat_model.named_steps['classifier'].feature_importances_
std = np.std([tree.feature_importances_ for tree in cat_model.named_steps['classifier'].estimators_], axis=0)
feature_names = [f"{x}" for x in cat_model.named_steps["preprocessor"].get_feature_names_out()]
forest_importances = pd.DataFrame({"feature": feature_names, "importance": importances, "std": std}).sort_values("importance", ascending=False)

fig = px.bar(forest_importances[forest_importances["importance"] > 0.01],x="feature",y="importance", error_y="std", title="Feature importances using MDI")
fig.show()

## 3rd Model: Feature Combination


For the third model we will combine some features.
First ones are: features related to browser will be merged, features related to device will be merged together and (Protocol,Packet Type,Scale Packet Length) will be merged (from the features analysis it was the one with the higher mean between min max of each combination) <br>
Anomaly score will be multiply by itself

In [18]:
Xmer_dataset = X_dataset.copy()
Xmer_dataset["Browser"] = Xmer_dataset["Browser Name"] + "_" + Xmer_dataset["Browser Version"].astype(str) + "_" + Xmer_dataset["Browser Minor"].astype(str)
Xmer_dataset = Xmer_dataset.drop(columns=["Browser Name", "Browser Version", "Browser Minor"])
Xmer_dataset["Device"] = Xmer_dataset["Device Brand"] + "_" + Xmer_dataset["Device Model"]
Xmer_dataset = Xmer_dataset.drop(columns=["Device Brand", "Device Model"])
columns = ["Year", "Month", "Day", "Hour", "Minute", "Second", "DayOfWeek"]
Xmer_dataset[columns] = Xmer_dataset[columns].astype("category")
Xmer_dataset["Syn_feature_1"] = Xmer_dataset["Protocol"] + "_" + Xmer_dataset["Packet Type"] + Xmer_dataset["Traffic Type"]
Xmer_dataset = Xmer_dataset.drop(columns=["Protocol", "Packet Type", "Traffic Type"])
Xmer_dataset["Syn_feature_2"] = Xmer_dataset["Malware Indicators"] + "_" + Xmer_dataset["Alerts/Warnings"] + "_" + Xmer_dataset["IDS/IPS Alerts"]
Xmer_dataset = Xmer_dataset.drop(columns=["Malware Indicators", "Alerts/Warnings", "IDS/IPS Alerts"])
Xmer_dataset["Syn_feature_3"] = Xmer_dataset["Anomaly Scores"]**2
Xmer_dataset = Xmer_dataset.drop(columns=["Anomaly Scores"])


### Pipeline Defition

In [19]:
cat_cols = Xmer_dataset.select_dtypes(include=["str", "category"]).columns
print("Categorical columns:", cat_cols.tolist())
bool_cols = Xmer_dataset.select_dtypes(include=["bool"]).columns
print("Boolean columns:", bool_cols.tolist())
num_cols = Xmer_dataset.select_dtypes(include=["int64", "float64"]).columns
print("Numerical columns:", num_cols.tolist())
passthrough_cols = [col for col in Xmer_dataset.columns if col not in cat_cols and col not in bool_cols and col not in num_cols]

numeric_transformer = Pipeline(
        steps = [("scaler", StandardScaler())]
    )

categorical_transformer = Pipeline([
        ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant", fill_value="unknown")),
        ("encoder", OneHotEncoder(handle_unknown="ignore", drop="first"))
        ])
boolean_transformer = Pipeline([
        ("encoder", TargetEncoder(target_type="binary")),
        ])

preprocessor = ColumnTransformer(
        transformers=[
            ("cat", categorical_transformer, cat_cols),
            ("num", numeric_transformer, num_cols),
            ("bool", boolean_transformer, bool_cols)
        ]
    )

mer_model = Pipeline([
        ("preprocessor", preprocessor),
        ("classifier", RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE, n_jobs=6, verbose=1))
    ])

Categorical columns: ['Attack Signature', 'Action Taken', 'Severity Level', 'Network Segment', 'Firewall Logs', 'Log Source', 'Year', 'Month', 'Day', 'Hour', 'Minute', 'Second', 'DayOfWeek', 'OS Name', 'Packet Bin', 'Browser', 'Device', 'Syn_feature_1', 'Syn_feature_2']
Boolean columns: ['Is Proxy', 'Global Source IP', 'Global Destination IP']
Numerical columns: ['Source Port', 'Destination Port', 'Packet Length', 'IsWeekend', 'Int Source IP', 'Int Destination IP', 'Syn_feature_3']


### Model Fitting and Result

In [20]:
X_train, X_test, y_train, y_test = train_test_split(Xmer_dataset, Y_true, test_size=0.2, stratify=Y_true, random_state=RANDOM_STATE)
start_time = monotonic()
mer_model.fit(X_train, y_train)
print("Time taken to fit the model: %.2f seconds" % (monotonic() - start_time))
print("Model score: %.3f" % mer_model.score(X_test, y_test))
y_pred = mer_model.predict(X_test)
print(classification_report(y_pred, y_test, target_names=labels))

[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    2.7s
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed:    6.5s finished


Time taken to fit the model: 6.75 seconds
Model score: 0.333
              precision    recall  f1-score   support

        DDoS       0.36      0.33      0.35      2893
   Intrusion       0.33      0.33      0.33      2607
     Malware       0.31      0.33      0.32      2500

    accuracy                           0.33      8000
   macro avg       0.33      0.33      0.33      8000
weighted avg       0.33      0.33      0.33      8000



[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed:    0.0s finished


### Feature Importance

In [21]:
importances = mer_model.named_steps['classifier'].feature_importances_
std = np.std([tree.feature_importances_ for tree in mer_model.named_steps['classifier'].estimators_], axis=0)
feature_names = [f"{x}" for x in mer_model.named_steps["preprocessor"].get_feature_names_out()]
forest_importances = pd.DataFrame({"feature": feature_names, "importance": importances, "std": std}).sort_values("importance", ascending=False)
fig = px.bar(forest_importances[forest_importances["importance"] > 0.01],x="feature",y="importance", error_y="std", title="Feature importances using MDI")
fig.show()

# Gradient Boosting Models

## Simple GradientBoosting Model

### Pipeline Definition

In [22]:
cat_cols = Xmer_dataset.select_dtypes(include=["str", "category"]).columns
print("Categorical columns:", cat_cols.tolist())
bool_cols = Xmer_dataset.select_dtypes(include=["bool"]).columns
print("Boolean columns:", bool_cols.tolist())
num_cols = Xmer_dataset.select_dtypes(include=["int64", "float64"]).columns
print("Numerical columns:", num_cols.tolist())
passthrough_cols = [col for col in Xmer_dataset.columns if col not in cat_cols and col not in bool_cols and col not in num_cols]

numeric_transformer = Pipeline(
        steps = [("scaler", StandardScaler())]
    )

categorical_transformer = Pipeline([
        ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant", fill_value="unknown")),
        ("encoder", OneHotEncoder(handle_unknown="ignore", drop="first"))
        ])
boolean_transformer = Pipeline([
        ("encoder", TargetEncoder(target_type="binary")),
        ])

preprocessor = ColumnTransformer(
        transformers=[
            ("cat", categorical_transformer, cat_cols),
            ("num", numeric_transformer, num_cols),
            ("bool", boolean_transformer, bool_cols)
        ]
    )

grad_model = Pipeline([
        ("preprocessor", preprocessor),
        ("classifier", GradientBoostingClassifier(n_estimators=300, max_depth = None, min_samples_split=30, random_state=RANDOM_STATE, n_iter_no_change=15))
    ])


Categorical columns: ['Attack Signature', 'Action Taken', 'Severity Level', 'Network Segment', 'Firewall Logs', 'Log Source', 'Year', 'Month', 'Day', 'Hour', 'Minute', 'Second', 'DayOfWeek', 'OS Name', 'Packet Bin', 'Browser', 'Device', 'Syn_feature_1', 'Syn_feature_2']
Boolean columns: ['Is Proxy', 'Global Source IP', 'Global Destination IP']
Numerical columns: ['Source Port', 'Destination Port', 'Packet Length', 'IsWeekend', 'Int Source IP', 'Int Destination IP', 'Syn_feature_3']


### Model fitting and result

In [23]:
X_train, X_test, y_train, y_test = train_test_split(Xmer_dataset, Y_true, test_size=0.2, stratify=Y_true, random_state=RANDOM_STATE)
start_time = monotonic()
grad_model.fit(X_train, y_train)
print("Time taken to fit the model: %.2f seconds" % (monotonic() - start_time))
print("Model score: %.3f" % grad_model.score(X_test, y_test))
y_pred = grad_model.predict(X_test)
print(classification_report(y_pred, y_test, target_names=labels))

Time taken to fit the model: 213.74 seconds
Model score: 0.335
              precision    recall  f1-score   support

        DDoS       0.34      0.33      0.33      2772
   Intrusion       0.33      0.33      0.33      2710
     Malware       0.33      0.35      0.34      2518

    accuracy                           0.33      8000
   macro avg       0.33      0.34      0.33      8000
weighted avg       0.33      0.33      0.33      8000



### Feature Importance

In [24]:
importances = grad_model.named_steps['classifier'].feature_importances_
forests = [forest for forest in grad_model.named_steps['classifier'].estimators_]
std = np.std([tree.feature_importances_ for forest in forests for tree in forest], axis=0)
feature_names = [f"{x}" for x in grad_model.named_steps["preprocessor"].get_feature_names_out()]
forest_importances = pd.DataFrame({"feature": feature_names, "importance": importances, "std": std}).sort_values("importance", ascending=False)

fig = px.bar(forest_importances[forest_importances["importance"] > 0.01],x="feature",y="importance", error_y="std", title="Feature importances using MDI")
fig.show()

## HistGradientBoost Classifier

No need to use one hot encoder for this model

In [25]:
Xhist_dataset = X_dataset.copy()

### Pipeline Definition

In [26]:
cat_cols = Xhist_dataset.select_dtypes(include=["str", "category","bool", "object"]).columns.to_list()
print("Categorical columns:", cat_cols)
num_cols = Xhist_dataset.select_dtypes(include=["number"]).columns
passthrough_cols = [col for col in Xhist_dataset.columns if col not in cat_cols and col not in bool_cols and col not in num_cols]

# _tmp = SimpleImputer(strategy="constant", fill_value="unknown").fit_transform(Xhist_dataset[cat_cols])
# _tmp_ohe = OneHotEncoder(drop="first", handle_unknown="ignore").fit(_tmp)
# all_categories = _tmp_ohe.categories_


numeric_transformer = Pipeline(
        steps = [("scaler", StandardScaler())]
    )
# numeric_transformer.fit(Xhist_dataset[num_cols], Y_true)
# Xhist_dataset[num_cols] = numeric_transformer.transform(Xhist_dataset[num_cols])

preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, num_cols),
            ("cat", "passthrough", cat_cols)
        ]
    )

# Calculate indices of categorical features after preprocessing
# num_cols come first, then cat_cols
cat_feature_indices = range(len(num_cols), len(num_cols) + len(cat_cols))

hist_model = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", HistGradientBoostingClassifier(max_iter=300, max_depth=None, random_state=RANDOM_STATE, categorical_features=cat_feature_indices, verbose=1))
])

Categorical columns: ['Protocol', 'Packet Type', 'Traffic Type', 'Malware Indicators', 'Alerts/Warnings', 'Attack Signature', 'Action Taken', 'Severity Level', 'Network Segment', 'Firewall Logs', 'IDS/IPS Alerts', 'Log Source', 'Browser Name', 'OS Name', 'Device Brand', 'Device Model', 'Is Proxy', 'Global Source IP', 'Global Destination IP', 'Packet Bin']


### Model Fitting

In [27]:
X_train, X_test, y_train, y_test = train_test_split(Xhist_dataset, Y_true, test_size=0.2, stratify=Y_true, random_state=RANDOM_STATE)
start_time = monotonic()
hist_model.fit(X_train, y_train)
print("Time taken to fit the model: %.2f seconds" % (monotonic() - start_time))
print("Model score: %.3f" % hist_model.score(X_test, y_test))
y_pred = hist_model.predict(X_test)
print(classification_report(y_pred, y_test, target_names=labels))

Binning 0.008 GB of training data: 0.020 s
Binning 0.001 GB of validation data: 0.001 s
Fitting gradient boosted rounds:
Fit 33 trees in 0.240 s, (1023 total leaves)
Time spent computing histograms: 0.044s
Time spent finding best splits:  0.018s
Time spent applying splits:      0.015s
Time spent predicting:           0.002s
Time taken to fit the model: 0.28 seconds
Model score: 0.332
              precision    recall  f1-score   support

        DDoS       0.35      0.33      0.34      2872
   Intrusion       0.30      0.33      0.32      2410
     Malware       0.34      0.33      0.34      2718

    accuracy                           0.33      8000
   macro avg       0.33      0.33      0.33      8000
weighted avg       0.33      0.33      0.33      8000



# Model with Less features

## Model with no combination

We try to model using less features. All features related to post analysis are ommitted: we keep information related to IP, port packet length and month day as well as dayofweek

In [28]:
columns = ["Int Source IP", "Int Destination IP", "Protocol", "Packet Type", "Traffic Type", "Anomaly Scores", "Severity Level", "Month", "DayOfWeek"]
Xsimple_dataset = X_dataset[columns].copy()
Xsimple_dataset[["Month", "DayOfWeek"]] = Xsimple_dataset[["Month", "DayOfWeek"]].astype("category")

### Pipeline Definition

In [29]:
cat_cols = Xsimple_dataset.select_dtypes(include=["str", "object"]).columns
print("Categorical columns:", cat_cols.tolist())
bool_cols = Xsimple_dataset.select_dtypes(include=["bool"]).columns
print("Boolean columns:", bool_cols.tolist())
num_cols = Xsimple_dataset.select_dtypes(include=["number"]).columns
print("Numerical columns:", num_cols.tolist())
passthrough_cols = [col for col in Xsimple_dataset.columns if col not in cat_cols and col not in bool_cols and col not in num_cols]

numeric_transformer = Pipeline(
        steps = [("scaler", StandardScaler())]
    )

categorical_transformer = Pipeline([
        ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant", fill_value="unknown")),
        ("encoder", OneHotEncoder(handle_unknown="ignore", drop="first"))
        ])

boolean_transformer = Pipeline([
        ("encoder", TargetEncoder(target_type="binary")),
        ])

preprocessor = ColumnTransformer(
        transformers=[
            ("cat", categorical_transformer, cat_cols),
            ("num", numeric_transformer, num_cols),
            ("bool", boolean_transformer, bool_cols)
        ])

param_grid = {
    "classifier__n_estimators": [100, 300],
    "classifier__max_depth": [None, 10, 20],
    "classifier__min_samples_split": [2, 10, 30]
}

model = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=6, verbose=1))
    ])

Categorical columns: ['Protocol', 'Packet Type', 'Traffic Type', 'Severity Level']
Boolean columns: []
Numerical columns: ['Int Source IP', 'Int Destination IP', 'Anomaly Scores']


### Model Fitting and Result

In [30]:
X_train, X_test, y_train, y_test = train_test_split(Xsimple_dataset, Y_true, test_size=0.2, stratify=Y_true, random_state=RANDOM_STATE)
start_time = monotonic()
search = GridSearchCV(model, param_grid)
search.fit(X_train, y_train)
print("Time taken to fit the model: %.2f seconds" % (monotonic() - start_time))
print("Best parameters: ", search.best_params_)
print("Model score: %.3f" % search.score(X_test, y_test))
y_pred = search.predict(X_test)
print(classification_report(y_pred, y_test, target_names=labels))

[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.3s
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed:    0.7s finished
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.3s
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed:    0.7s finished
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.3s
[Parallel(n_job

Time taken to fit the model: 125.44 seconds
Best parameters:  {'classifier__max_depth': None, 'classifier__min_samples_split': 30, 'classifier__n_estimators': 100}
Model score: 0.334
              precision    recall  f1-score   support

        DDoS       0.35      0.35      0.35      2714
   Intrusion       0.32      0.33      0.33      2589
     Malware       0.32      0.32      0.32      2697

    accuracy                           0.33      8000
   macro avg       0.33      0.33      0.33      8000
weighted avg       0.33      0.33      0.33      8000



[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed:    0.8s finished
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed:    0.0s finished


## Model with feature combination

In [31]:
columns = ["Int Source IP", "Int Destination IP", "Protocol", "Packet Type", "Traffic Type", "Anomaly Scores", "Severity Level", "Month", "DayOfWeek"]
Xsm_dataset = X_dataset[columns].copy()
Xsm_dataset["Syn Feature 1"] = Xsm_dataset["Protocol"] + "_" + Xsm_dataset["Packet Type"] + Xsm_dataset["Traffic Type"]
Xsm_dataset = Xsm_dataset.drop(columns=["Protocol", "Packet Type", "Traffic Type"])
Xsm_dataset["Syn Feature 2"] = Xsm_dataset["Anomaly Scores"]**2
Xsm_dataset = Xsm_dataset.drop(columns=["Anomaly Scores"])
Xsm_dataset["Syn Feature 3"] = Xsm_dataset["Month"].astype(str) + "_" + Xsm_dataset["DayOfWeek"].astype(str)
Xsm_dataset = Xsm_dataset.drop(columns=["Month", "DayOfWeek"])

### Pipeline Definition

In [32]:
cat_cols = Xsm_dataset.select_dtypes(include=["str", "object"]).columns
print("Categorical columns:", cat_cols.tolist())
bool_cols = Xsm_dataset.select_dtypes(include=["bool"]).columns
print("Boolean columns:", bool_cols.tolist())
num_cols = Xsm_dataset.select_dtypes(include=["number"]).columns
print("Numerical columns:", num_cols.tolist())
passthrough_cols = [col for col in Xsm_dataset.columns if col not in cat_cols and col not in bool_cols and col not in num_cols]

numeric_transformer = Pipeline(
        steps = [("scaler", StandardScaler())]
    )

categorical_transformer = Pipeline([
        ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant", fill_value="unknown")),
        ("encoder", OneHotEncoder(handle_unknown="ignore", drop="first"))
        ])
boolean_transformer = Pipeline([
        ("encoder", TargetEncoder(target_type="binary")),
        ])

preprocessor = ColumnTransformer(
        transformers=[
            ("cat", categorical_transformer, cat_cols),
            ("num", numeric_transformer, num_cols),
            ("bool", boolean_transformer, bool_cols)
        ]
    )
param_grid = {
    "classifier__n_estimators": [100, 200],
    "classifier__max_depth": [None, 10, 20],
    "classifier__min_samples_split": [2, 10, 30]
}


comb_model = Pipeline([
        ("preprocessor", preprocessor),
        ("classifier", RandomForestClassifier(n_estimators=300, random_state=RANDOM_STATE, n_jobs=6, verbose=1))
    ])

Categorical columns: ['Severity Level', 'Syn Feature 1', 'Syn Feature 3']
Boolean columns: []
Numerical columns: ['Int Source IP', 'Int Destination IP', 'Syn Feature 2']


### Model Fitting and Result

In [33]:
X_train, X_test, y_train, y_test = train_test_split(Xsm_dataset, Y_true, test_size=0.2, stratify=Y_true, random_state=RANDOM_STATE)
start_time = monotonic()
search = GridSearchCV(comb_model, param_grid)
search.fit(X_train, y_train)
print("Time taken to fit the model: %.2f seconds" % (monotonic() - start_time))
print("Best parameters: ", search.best_params_)
print("Model score: %.3f" % search.score(X_test, y_test))
y_pred = search.predict(X_test)
print(classification_report(y_pred, y_test, target_names=labels))

[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    1.5s
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed:    3.4s finished
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    1.3s
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed:    3.3s finished
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    1.5s
[Parallel(n_job

Time taken to fit the model: 160.44 seconds
Best parameters:  {'classifier__max_depth': None, 'classifier__min_samples_split': 10, 'classifier__n_estimators': 100}
Model score: 0.331
              precision    recall  f1-score   support

        DDoS       0.34      0.33      0.34      2719
   Intrusion       0.33      0.33      0.33      2651
     Malware       0.33      0.33      0.33      2630

    accuracy                           0.33      8000
   macro avg       0.33      0.33      0.33      8000
weighted avg       0.33      0.33      0.33      8000



[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed:    0.0s finished
